# DCGAN

### ToDo
* DCGAN
    * MNIST 에 맞게 구현하였는데 실제로 논문에서는 어떻게?
    * keras MNIST 데이터셋 체크
    * Adam-a/b, init 등 다른 파라메터 체크
* Tensorboard with summary
* 모델이 잘 작동하는지 확인해보고싶다.
    * 특히, BN 의 모멘텀을 확인해 볼 수 있나?
* Other datasets
    * Keras 에서 다른 데이터셋도 지원하는듯?
* Refactoring with model.py (ipynb 에 몰빵한걸 분리)

### Default
* 기본적으로 이미지를 [-1, 1] 로 normalize

### Discriminator
![discriminator](http://bamos.github.io/data/2016-08-09/discrim-architecture.png)

* Activation function: Leaky ReLU
* BN 은 첫번째 layer 에는 붙지 않음
    * Generator 가 생성한 sample 의 statistics 를 잘 학습하기 위함이라고 함

### Generator
![generator](http://bamos.github.io/data/2016-08-09/gen-architecture.png)

* Activation function: ReLU / 마지막 레이어만 tanh
    * 기본적으로 이미지를 [-1, 1] 로 normalize 하기때문에 tanh results 가 바로 output 이 됨
* z_dim 은 기본적으로는 100인듯. 첫번째에서는 FC 로 차원수를 알맞게 늘린 후 reshape.
* BN 은 output layer 에는 붙지 않음 & FC layer 로 project 할때도 붙지 않음

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [ ]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)

## DCGAN for MNIST

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis=('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    
    return fig

In [ ]:
def lrelu(x, leak=0.2):
    return tf.maximum(x, x*leak)

In [ ]:
def discriminator(x, reuse=False):
    # x: [-1, 28, 28, 1]
    with tf.variable_scope("discriminator"):
        with tf.variable_scope("D_conv1"):
            h1_conv = tf.layers.conv2d(inputs=x, filters=64, kernel_size=[5, 5], strides=2, 
                                       padding='SAME', use_bias=True, reuse=reuse)
            h1 = lrelu(h1_conv) # [-1, 14, 14, 64]
        with tf.variable_scope("D_conv2"):
            h2_conv = tf.layers.conv2d(inputs=h1, filters=128, kernel_size=[5, 5], strides=2, 
                                       padding='SAME', use_bias=False, reuse=reuse)
            h2_bn = tf.layers.batch_normalization(h2_conv, training=training, reuse=reuse)
            h2 = lrelu(h2_bn) # [-1, 7, 7, 128]
        with tf.variable_scope("D_dense"):
            logits = tf.layers.dense(inputs=h2, units=1, name="D_logits", reuse=reuse)
            prob = tf.sigmoid(logits, "D_prob")
        
        return prob, logits

In [ ]:
def generator(z):
    # z: [-1, 100]
    # 128 -> 64 -> output
    with tf.variable_scope("generator"):
        with tf.variable_scope("G_project_reshape"):
            h1_dense = tf.layers.dense(z, 128*7*7, activation=tf.nn.relu)
            h1 = tf.reshape(h1_dense, [-1, 7, 7, 128])
        with tf.variable_scope("G_conv_T1"): 
            h2_conv_T = tf.layers.conv2d_transpose(h1, 64, [5,5], strides=2, padding='SAME', use_bias=False)
            h2_bn = tf.layers.batch_normalization(h2_conv_T, training=training)
            h2 = tf.nn.relu(h2_bn) # [-1, 14, 14, 64]
        with tf.variable_scope("G_conv_T2"): 
            h3_conv_T = tf.layers.conv2d_transpose(h2, 1, [5,5], strides=2, padding='SAME', use_bias=False)
#             h3_bn = tf.layers.batch_normalization(h3_conv_T, training=training)
#             h3 = tf.nn.tanh(h3_conv_T) # [-1, 28, 28, 1]
            h3 = tf.nn.sigmoid(h3_conv_T)
            # mnist dataset 은 0~1 이므로, tanh 에 맞춰주기 위해 초기 normalization 을 해주기로 하자.
            # 라고 생각했지만 귀찮아서 그냥 시그모이드로 해주기로 하자.
            
        return h3

In [ ]:
def Z_sampler(m, n):
    # 가우시안이 더 좋다고 함
    return np.random.uniform(-1., 1., size=[m, n])
#     return tf.random_uniform([m, n], minval=-1., maxval=1.)

In [ ]:
tf.reset_default_graph()

# placeholders
X = tf.placeholder(tf.float32, [None, 784], name='X')
Z = tf.placeholder(tf.float32, [None, 100], name='Z')
training = tf.placeholder(tf.bool, name='training')

# calc loss
G_sample = generator(Z)
X_img = tf.reshape(X, [-1, 28, 28, 1])
D_prob_real, D_logit_real = discriminator(X_img)
D_prob_fake, D_logit_fake = discriminator(G_sample, reuse=True)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, 
                                                                     labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake,
                                                                     labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, 
                                                                labels=tf.ones_like(D_logit_fake)))
# solvers
D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

# TF 에서 batchnorm 을 쓰면 moving_mean / moving_variance 를 게산해야 함. 
# 이 두 변수를 업데이트하는 모멘텀 오퍼레이션이 GraphKeys.UPDATE_OPS 로 잡혀있대.
# 그래서 그냥 돌리면 학습과정에서 모멘텀 오퍼레이션이 작동을 안해서 업데이트가 안 됨. 
# 따라서, control_dependencies 를 사용해서 업데이트하도록 지정해 주어야 한다.
# 아래와 같이 control_dependencies 를 사용하면 D_solver 오퍼레이션이 작동하기 전에 D_update_ops 오퍼레이션이 먼저 작동하도록 지정해주는 것.
# https://www.facebook.com/groups/TensorFlowKR/permalink/447827285558335/
D_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='discriminator')
G_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='generator')
lr = 0.0002

with tf.control_dependencies(D_update_ops):
    D_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=D_vars)
with tf.control_dependencies(G_update_ops):
    G_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=G_vars)

In [ ]:
mnist.validation.num_examples

In [ ]:
epoch_n = 500
batch_size = 64
N = mnist.train.num_examples

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epoch_n):
    for _ in range(N // batch_size):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)

        # 'Z' 가 겹쳐도 상관 없을것 같다.
        feeds = {X: X_batch, Z: Z_sampler(batch_size, 100), training: True}
        _ = sess.run(D_solver, feeds)
        _ = sess.run(G_solver, feeds)
    
    # progress check
    # 1. loss check
    feeds[training] = False
    D_loss_cur = sess.run(D_loss, feeds)
    G_loss_cur = sess.run(G_loss, feeds)
    print("[{}/{}] D_loss: {:.4f} / G_loss: {:.4f}".format(epoch+1, epoch_n, D_loss_cur, G_loss_cur))
    
    # 2. sampling
    samples = sess.run(G_sample, {Z: Z_sampler(16, 100), training: False})
    fig = plot(samples)
    plt.show()